In [ ]:
#@title Install updated Transformers library
%%capture
!pip install git+https://github.com/AlekseyKorshuk/transformers.git@opt


# Prepare Meta's model

In [ ]:
#@title Download weights
!mkdir opt-model
!wget https://dl.fbaipublicfiles.com/opt/v1_20220502/350m/reshard.pt

--2022-05-05 20:43:11--  https://dl.fbaipublicfiles.com/opt/v1_20220502/350m/reshard.pt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.75.142, 104.22.74.142, 172.67.9.4, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.75.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 714040635 (681M) [binary/octet-stream]
Saving to: ‘reshard.pt’

reshard.pt          100%[===================>] 680.96M  20.4MB/s    in 34s     

2022-05-05 20:43:46 (20.1 MB/s) - ‘reshard.pt’ saved [714040635/714040635]



In [ ]:
#@title Convert Pytorch state dict
import torch
initials = [
            torch.load("/content/reshard.pt"),
            # torch.load("/content/reshard-model_part-0.pt"),
            # torch.load("/content/reshard-model_part-1.pt")
           ]
state_dict = {}
for initial in initials:
  for key in initial.keys():
    if isinstance(initial[key], dict):
      for key2 in initial[key].keys():
        state_dict[f"{key}.{key2}"] = initial[key][key2]
    else:
      state_dict[f"{key}"] = initial[key]

torch.save(state_dict, "opt-model/pytorch_model.bin")

In [ ]:
#@title Load config file
config_dict = {
  "_name_or_path": "test",
  "activation_dropout": 0.0,
  "activation_function": "gelu_new",
  "architectures": [
    "OPTForCausalLM"
  ],
  "attention_dropout": 0.0,
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 24,
  "decoder_start_token_id": 2,
  "do_sample": True,
  "dropout": 0.1,
  "embd_pdrop": 0.1,
  "embed_dim": 512,
  "eos_token_id": 50256,
  "forced_eos_token_id": 2,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "initializer_range": 0.02,
  "is_decoder": True,
  "is_encoder_decoder": False,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_epsilon": 1e-05,
  "max_length": 50,
  "max_position_embeddings": 2048,
  "model_type": "opt",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_layer": 12,
  "n_positions": 1024,
  "pad_token_id": 1,
  "resid_pdrop": 0.1,
  "scale_embedding": False,
  "share_input_output_embed": False,
  "summary_activation": None,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": True,
  "summary_type": "cls_index",
  "summary_use_proj": True,
  "task_specific_params": {
    "text-generation": {
      "do_sample": True,
      "max_length": 50
    }
  },
  "torch_dtype": "float32",
  "transformers_version": "4.19.0.dev0",
  "use_cache": False,
  "vocab_size": 50272
}

import json
with open('opt-model/config.json', 'w') as fp:
    json.dump(config_dict, fp)

# Inference

## With early prepared model

In [ ]:
from transformers import pipeline, OPTForCausalLM, GPT2Tokenizer
model = OPTForCausalLM.from_pretrained("opt-model").eval()
tokenizer = GPT2Tokenizer.from_pretrained("AlekseyKorshuk/opt-350m")
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

results = pipe("Today is a beautiful day and I want to")
results

## With pipeline from my ready-to-go hub repo

In [ ]:
from transformers import pipeline
pipe = pipeline("text-generation", "AlekseyKorshuk/opt-350m")

results = pipe("Today is a beautiful day and I want to")
results